## Подключение библиотек и источников данных, подготовка к анализу

In [22]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [23]:
pd.io.sql.read_sql('select * from books limit 5', con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [24]:
pd.io.sql.read_sql('select count(*) from books', con = engine) 

,count
0,1000


In [25]:
pd.io.sql.read_sql('select * from authors limit 5', con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [26]:
pd.io.sql.read_sql('select count(*) from authors', con = engine) 

,count
0,636


In [27]:
pd.io.sql.read_sql('select * from publishers limit 5', con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [28]:
pd.io.sql.read_sql('select count(*) from publishers', con = engine) 

,count
0,340


In [29]:
pd.io.sql.read_sql('select * from ratings limit 5', con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [30]:
pd.io.sql.read_sql('select count(*) from ratings', con = engine) 

,count
0,6456


In [31]:
pd.io.sql.read_sql('select * from reviews limit 5', con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [32]:
pd.io.sql.read_sql('select count(*) from reviews', con = engine) 

,count
0,2793


## Задания

- [x] Посчитайте, сколько книг вышло после 1 января 2000 года;
- [x] Для каждой книги посчитайте количество обзоров и среднюю оценку;
- [x] Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- [x] Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- [x] Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.


In [33]:
pd.io.sql.read_sql('''
    select count(*) 
    from books
    where publication_date > DATE '2000-01-01' ''', con = engine) 

,count
0,819


После 1 января 2000 вышло 819 книг

In [34]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(pd.io.sql.read_sql('''
        select b.title,
            re.book_id,
            count(*),
            avg(ra.avg_rating)
        from books b
        left join reviews re on b.book_id=re.book_id 
        left join (select book_id,
            avg(rating) avg_rating
            from ratings
            group by book_id) ra on ra.book_id=re.book_id
        group by re.book_id, b.title
        ''', con = engine)) 

,title,book_id,count,avg
0,Fantastic Beasts and Where to Find Them (Hogwa...,238.0,4,3.888889
1,Lucy Sullivan Is Getting Married,412.0,2,2.500000
2,The Lost Continent: Travels in Small Town America,786.0,2,3.500000
3,Wild Magic (Immortals #1),981.0,2,4.333333
4,Last Man Standing,385.0,2,3.500000
5,Starship Troopers,595.0,3,4.500000
6,Memoirs of a Geisha,427.0,3,3.700000
7,The Bourne Ultimatum (Jason Bourne #3),662.0,2,4.000000
8,Body Double (Rizzoli & Isles #4),120.0,2,4.000000
9,Gulliver's Travels,291.0,3,3.142857


In [35]:
pd.io.sql.read_sql('''
    select p.publisher,
        count(*) cnt
    from publishers p
    join (select title,
        book_id,
        publisher_id
    from books
    where num_pages > 50) b on b.publisher_id=p.publisher_id 
    group by p.publisher
    order by cnt desc
    limit 1
    ''', con = engine) 

,publisher,cnt
0,Penguin Books,42


Больше всего книг выпустило издание Penguin Books

In [36]:
pd.io.sql.read_sql('''
    select a.author,
        r.avg_rating
    from books b
    join (select 
        book_id,
        avg(rating) avg_rating
    from ratings
    group by book_id
    having count(book_id) > 50) r on b.book_id=r.book_id 
    join authors a on a.author_id=b.author_id
    group by a.author, r.avg_rating
    order by r.avg_rating desc
    limit 1    
    ''', con = engine) 

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.414634


In [37]:
pd.io.sql.read_sql('''
    select a.author,
        avg(r.avg_rating)
    from (select 
        book_id,
        avg(rating) avg_rating
    from ratings
    group by book_id
    having count(book_id) > 50) r
    left join books b on b.book_id=r.book_id
    left join authors a on a.author_id=b.author_id
    group by a.author
    order by avg(r.avg_rating) desc
    limit 1    
    ''', con = engine) 

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844


Самая высокая средняя оценка (4.41) для авторов, по книгам, которые набрали больше 50 оценок у автора J.K. Rowling/Mary GrandPré

In [38]:
pd.io.sql.read_sql('''
    select round(avg(cnt))
    from (select re.username,
        count(*) cnt
    from reviews re
    join (select 
        username,
        count(*) cnt
    from ratings
    group by username
    having count(*) > 50) ra on ra.username=re.username 
    group by re.username
    order by cnt desc) t
    ''', con = engine) 

,round
0,24.0


В среднем, пользователи у которых больше 50 оценок, оставляют 24 отзывов

## Выводы

Предобработка данных не требуется

Исследовательский анализ данных (с использованием SQL)
- После 1 января 2000 вышло 819 книг
- Больше всего книг (42) выпустило издание Penguin Books
- Самая высокая средняя оценка (4.41) для авторов, по книгам, которые набрали больше 50 оценок у автора J.K. Rowling/Mary GrandPré
- В среднем, пользователи у которых больше 50 оценок, оставляют 24 отзывов
- Получена сводная таблица, в которой для каждой книги есть информация о количестве обзоров и средней оценке, с таблицей можно ознакомиться в исследовании